#Prediction of Next word- Model

In [37]:
#modules required for EDA
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
import tensorflow.keras.backend as K
from keras.layers import LSTM
from keras.callbacks import LambdaCallback
from keras.layers import Dense, Activation

In [2]:
from google.colab import files
uploaded = files.upload()

Saving corpus.txt to corpus.txt


In [13]:
import pandas as pd
import io
true_data = pd.read_csv(io.BytesIO(uploaded['corpus.txt']))
with open('corpus.txt') as subject:
  cache = subject.readlines()
translator = str.maketrans('', '', string.punctuation) # Remove punctuation
lines = [doc.lower().translate(translator) for doc in cache] # Switch to lower case

In [14]:
print(lines[0][:100])

in science and engineering intelligent processing of complex signals such as images sound or languag


In [15]:
len(lines)

7200

In [16]:
# Generate an list of single/independent words

vocabulary = list(set(' '.join(lines).replace('\n','').split(' ')))
primary_store = {}
for strings, texts in enumerate(vocabulary):
  primary_store[texts] = strings

In [17]:
# PREVIEW OUTPUT ::
print(vocabulary[:50])
len(vocabulary)

['', 'predicting', 'stacked', 'distributing', 'lowerbounded', 'generally', 'depend', 'top', 'retaining', 'line', 'highly', 'explanation', 'certain', 'inputtohidden', 'postsynaptic', 'projection', 'topology', 'provable', 'team', 'empirical', 'resulting', 'cudasupport', 'interpretable', 'show', 'schemes', 'mature', 'prone', 'compiler', 'determining', 'nmf', 'evaluation', 'fast', 'static', 'controlled', 'brand', 'compress', 'term', 'hebbian', 'allows', 'do', 'raw', 'posteriori', 'community', 'adaption', 'devastating', 'parameterization', 'caffe', '1996', 'reidentification', 'advantages']


2694

In [20]:
# Splitting data into Train sets and test sets

X = [] 
y = []

for c in lines:
  xxxx = c.replace('\n','').split(' ')
  X.append(' '.join(xxxx[:-1])) # X from the corpus

  yyyy = [0 for i in range(len(vocabulary))] # Generate Y from the Vocabulary
  # yyyy[primary_store[xxxx[-1]]] = 1
  yyyy[primary_store[xxxx[-1]]] = 1
  y.append(yyyy)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
y_test = numpy.array(y_test)
y_train = numpy.array(y_train)

In [43]:
print(X_train[:10])
print(y_train[:10])
print(X_test[:10])
print(y_test[:10])

[[-0.03771037 -0.06771192 -0.05370538 ...  0.06933319 -0.00784063
  -0.05528936]
 [ 0.01183542 -0.06317    -0.0010509  ...  0.06420518 -0.05883576
  -0.02919188]
 [-0.03916729 -0.03120757 -0.05427597 ...  0.06412318 -0.06295582
  -0.06137133]
 ...
 [ 0.03922532 -0.06610486 -0.06339797 ...  0.07350147 -0.03273619
  -0.02008303]
 [ 0.00030299 -0.05887231 -0.02081317 ...  0.06502399 -0.0582498
  -0.0500095 ]
 [ 0.05783226 -0.06084208 -0.03166341 ...  0.06078244 -0.05897182
  -0.0489399 ]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[-0.06120554 -0.04776438 -0.05084328 ...  0.06391488 -0.0510295
  -0.06147793]
 [ 0.05910347 -0.0636731  -0.06219529 ...  0.05805291 -0.04416828
  -0.05484251]
 [-0.04787395 -0.04063858 -0.06472678 ...  0.06678558 -0.06455693
  -0.06263315]
 ...
 [-0.05151311 -0.04795229 -0.03584861 ...  0.06181872 -0.03392373
  -0.0543977 ]
 [ 0.05783226 -0.06084208 -0.03166341 ...  0.06078244 -0.0589

In [44]:
# Import the Universal Sentence Encoder's TF Hub module (Here we're making use of version 4)

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"  
appreciate = hub.load(module_url)

# Function for embedding
def embed(goodness):
  return appreciate(goodness)


In [45]:
appreciate.variables

ListWrapper([<tf.Variable 'Embeddings/sharded_0:0' shape=(26667, 320) dtype=float32, numpy=
array([[-0.44756386, -0.7523224 , -2.8879747 , ..., -4.275423  ,
        -0.50384414, -2.5144944 ],
       [ 0.19551526,  0.18014883,  0.24043915, ..., -0.2736297 ,
         0.10923431,  0.20877579],
       [-0.14282258,  0.0094786 , -0.02283357, ...,  0.05814617,
         0.07281963,  0.1362249 ],
       ...,
       [ 0.25273287, -0.06458262, -0.0383645 , ..., -0.05067257,
        -0.04441866,  0.03095888],
       [-0.01667571,  0.05448845,  0.009569  , ...,  0.00490127,
        -0.04033831,  0.24144703],
       [ 0.03673933, -0.1211024 , -0.03498175, ...,  0.07811887,
        -0.00703663,  0.22115262]], dtype=float32)>, <tf.Variable 'Embeddings/sharded_1:0' shape=(26667, 320) dtype=float32, numpy=
array([[ 3.1123035e+00,  1.9035040e+00, -2.4504054e+00, ...,
        -3.6000068e+00,  1.6262866e+00,  3.8986406e+00],
       [ 3.1896163e-02,  4.0815596e-02,  3.2115210e-02, ...,
        -8.2207076e-

In [24]:
# Wrapping up with the U-S-E

X_train = embed(X_train)
X_test = embed(X_test)
X_train = X_train.numpy()
X_test = X_test.numpy()

In [46]:
print(X_train[:10])
print(y_train[:10])
print(X_test[:10])
print(y_test[:10])
print(X_train.shape, X_test.shape, y_test.shape, y_train.shape)

[[-0.03771037 -0.06771192 -0.05370538 ...  0.06933319 -0.00784063
  -0.05528936]
 [ 0.01183542 -0.06317    -0.0010509  ...  0.06420518 -0.05883576
  -0.02919188]
 [-0.03916729 -0.03120757 -0.05427597 ...  0.06412318 -0.06295582
  -0.06137133]
 ...
 [ 0.03922532 -0.06610486 -0.06339797 ...  0.07350147 -0.03273619
  -0.02008303]
 [ 0.00030299 -0.05887231 -0.02081317 ...  0.06502399 -0.0582498
  -0.0500095 ]
 [ 0.05783226 -0.06084208 -0.03166341 ...  0.06078244 -0.05897182
  -0.0489399 ]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[-0.06120554 -0.04776438 -0.05084328 ...  0.06391488 -0.0510295
  -0.06147793]
 [ 0.05910347 -0.0636731  -0.06219529 ...  0.05805291 -0.04416828
  -0.05484251]
 [-0.04787395 -0.04063858 -0.06472678 ...  0.06678558 -0.06455693
  -0.06263315]
 ...
 [-0.05151311 -0.04795229 -0.03584861 ...  0.06181872 -0.03392373
  -0.0543977 ]
 [ 0.05783226 -0.06084208 -0.03166341 ...  0.06078244 -0.0589

# **Building the model**

In [35]:
model = Sequential()
# model.add(Embedding(input_dim=len(vocabulary), output_dim=100))
model = Sequential()
# model.add(LSTM(units=100, input_shape=[512]))
model.add(Dense(512, input_shape=[512], activation = 'relu'))
model.add(Dense(units=len(vocabulary), activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 2694)              1382022   
                                                                 
Total params: 1,644,678
Trainable params: 1,644,678
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Training the model. 

model.fit(X_train, y_train, batch_size=512, shuffle=True, epochs=20, validation_data=(X_test, y_test), callbacks=[LambdaCallback()])

Epoch 1/20
11/11 [==============================] - 4s 311ms/step - loss: 7.6593 - acc: 0.0983 - val_loss: 7.0400 - val_acc: 0.0661
Epoch 2/20
11/11 [==============================] - 2s 225ms/step - loss: 5.9810 - acc: 0.0748 - val_loss: 4.5621 - val_acc: 0.0778
Epoch 3/20
11/11 [==============================] - 2s 214ms/step - loss: 4.2465 - acc: 0.1085 - val_loss: 4.0874 - val_acc: 0.1194
Epoch 4/20
11/11 [==============================] - 2s 145ms/step - loss: 3.9884 - acc: 0.1837 - val_loss: 3.8765 - val_acc: 0.2478
Epoch 5/20
11/11 [==============================] - 2s 144ms/step - loss: 3.7651 - acc: 0.2013 - val_loss: 3.6225 - val_acc: 0.2389
Epoch 6/20
11/11 [==============================] - 2s 153ms/step - loss: 3.4731 - acc: 0.3300 - val_loss: 3.2970 - val_acc: 0.3822
Epoch 7/20
11/11 [==============================] - 2s 154ms/step - loss: 3.1168 - acc: 0.4994 - val_loss: 2.9075 - val_acc: 0.5856
Epoch 8/20
11/11 [==============================] - 2s 156ms/step - loss: 2.

In [47]:

# Creating function to predict and show detailed output

def next_word(collection=[], extent=1):

  for item in collection:
    text = item
    for i in range(extent):
      prediction = model.predict(x=embed([item]).numpy())
      idx = np.argmax(prediction[-1])
      item += ' ' + vocabulary[idx]
      
      print(text + ' --> ' + item + '\nNEXT WORD: ' + item.split(' ')[-1] + '\n')

In [48]:
# Testing the output

single_text = ['and some other essential']

next_word(single_text)

1/1 [==============================] - 0s 24ms/step
and some other essential --> and some other essential networks
NEXT WORD: networks



In [49]:
# Testing the output for a collection of words

text_collection = ['deep convolutional', 'simple and effective', 'a nonconvex', 'a']

next_word(text_collection)

1/1 [==============================] - 0s 28ms/step
deep convolutional --> deep convolutional networks
NEXT WORD: networks

1/1 [==============================] - 0s 25ms/step
simple and effective --> simple and effective acceleration
NEXT WORD: acceleration

1/1 [==============================] - 0s 22ms/step
a nonconvex --> a nonconvex benchmark
NEXT WORD: benchmark

1/1 [==============================] - 0s 23ms/step
a --> a accuracy
NEXT WORD: accuracy



In [42]:
# Storing data

vocabulary = numpy.array(vocabulary)
numpy.save('./vocabulary.npy', vocabulary)
model.save('./NWP-USE')